<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

**Importar arquivos de configuração do Kaglle**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [5]:
!kaggle datasets download tanyadayanand/geological-image-similarity
! unzip -qq geological-image-similarity

 51% 17.0M/33.6M [00:00<00:00, 46.3MB/s]
100% 33.6M/33.6M [00:00<00:00, 96.6MB/s]


### Data Preprocessing

In [4]:
andesite_dir = glob.glob('geological_similarity/andesite/*.jpg');gneiss_dir = glob.glob('geological_similarity/gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


5000
5000
4998
5000
5000


In [35]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [40]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']
andesite= class_names[0]
andesite

'andesite'

In [41]:
for i in andesite_dir[0:500]:
    andesite_dir.append([i,'class_name[0]'])
for j in gneiss_dir[0:400]:
   gneiss_dir.append([j,'class_name[1]])
df = andesite_dir + gneiss_dir
random.shuffle(df)
df

['geological_similarity/gneiss/LUVMT.jpg',
 'geological_similarity/gneiss/WN0KF.jpg',
 'geological_similarity/andesite/6Y4AQ.jpg',
 'geological_similarity/gneiss/X6D4S.jpg',
 'geological_similarity/gneiss/PVIOM.jpg',
 'geological_similarity/andesite/0P15I.jpg',
 'geological_similarity/gneiss/D4GRK.jpg',
 'geological_similarity/andesite/AQRN1.jpg',
 ['geological_similarity/andesite/GOCH6.jpg', 'class_name[0]'],
 ['geological_similarity/andesite/O1B1F.jpg'],
 'geological_similarity/gneiss/X6Z2I.jpg',
 ['geological_similarity/andesite/CTIFF.jpgclass_name[0]'],
 'geological_similarity/gneiss/KYQB8.jpg',
 ['geological_similarity/andesite/V4DUM.jpgclass_name[0]'],
 'geological_similarity/gneiss/VSHN7.jpg',
 'geological_similarity/gneiss/P0JCL.jpg',
 ['geological_similarity/gneiss/UN6D0.jpg'],
 'geological_similarity/andesite/YJD56.jpg',
 'geological_similarity/gneiss/IEPS9.jpg',
 'geological_similarity/gneiss/BSRSC.jpg',
 ['geological_similarity/andesite/TVWMI.jpg', 'class_name[0]'],
 'geolo

In [38]:
data_df = pd.DataFrame(df)
data_df

,0
0,geological_similarity/andesite/9FUK8.jpg
1,geological_similarity/andesite/20XR8.jpg
2,geological_similarity/andesite/PT0WZ.jpg
3,geological_similarity/andesite/Z7KXW.jpg
4,geological_similarity/andesite/AVLAI.jpg
...,...
12395,[geological_similarity/gneiss/ABJBS.jpg]
12396,[geological_similarity/gneiss/KNMTL.jpg]
12397,[geological_similarity/gneiss/06JLG.jpg]
12398,[geological_similarity/gneiss/VBEON.jpg]
